In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model

# Load model and data
model = load_model('lstm_model.h5')
data = pd.read_csv('final_btc_data.csv')

# Scale the data for evaluation
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['Close', 'Sentiment_MA', 'Close_MA']])

# Prepare sequences for evaluation
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i, 0])  # Predicting 'Close' price
    return np.array(X), np.array(y)

seq_length = 30
X, y = create_sequences(data_scaled, seq_length)

# Split data into testing set
split_idx = int(0.8 * len(X))
X_test, y_test = X[split_idx:], y[split_idx:]

# Predict on test set
y_pred = model.predict(X_test)

# Rescale predictions and actuals
y_test_rescaled = scaler.inverse_transform(np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 2)))))
y_pred_rescaled = scaler.inverse_transform(np.hstack((y_pred, np.zeros((y_pred.shape[0], 2)))))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_rescaled[:, 0], y_pred_rescaled[:, 0]))
print(f'RMSE: {rmse}')

# Plot predictions vs. actual values
plt.figure(figsize=(14, 5))
plt.plot(y_test_rescaled[:, 0], label='Actual Price', color='blue')
plt.plot(y_pred_rescaled[:, 0], label='Predicted Price', color='red')
plt.title('Bitcoin Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()
